# Usage Example



## Importing the NonPredictive class from the non-predictive module:

In [ ]:
import sys
sys.path.append('..')
from glsm.non_predictive.model import NonPredictive
from glsm.non_predictive.features import Feature

## Instanciate the model and add the features:

In [2]:
model = NonPredictive()

feature_a = Feature(
    name="Monthly Users",
    weight=0.5,
    points_map=[
        ["Up to 50K",0],
        ["50K - 100K",50],
        ["100K - 200K",75],
        ["More than 200K",100],
    ]
)
feature_b = Feature(
    name="Industry",
    weight=0.25,
    points_map=[
        ["Other",50 - (50/5)*5], # 0
        ["Agriculture",50 - (50/5)*4],
        ["Transportation",50 - (50/5)*3],
        ["Healthcare",50 - (50/5)*2],
        ["Manufacturing",50 - (50/5)],
        ["Education",50],
        ["Finance",50],
        ["Technology",50],
        ["Retail",50 + (50/2)],
        ["Telecom",50 + (50/2)*2], # 100
    ]
)

feature_c = Feature(
    name="Mkt Investment",
    weight=1,
    points_map=[
        ["Up to $50K",'0'],
        ["50k - $100K",30],
        ["100k - $200K",50],
        ["$200K - $300K",70],
        ["$300K - $400K",90],
        ["More than $400K",100],
    ]
)
model.add_features([feature_a, feature_b, feature_c])

## Read leads from a csv file and compute the lead score:

In [3]:
# or you can just use pandas instead
import pandas as pd

df = pd.read_csv('leads.csv')
df['lambda'] = df.apply(
    lambda row: model.compute_lambda(row.to_dict()),
    axis=1
)
df

,Monthly Users,Industry,Mkt Investment,lambda
0,50K - 100K,Technology,$300K - $400K,80.48
1,50K - 100K,Retail,100k - $200K,51.19
2,Up to 50K,Agriculture,50k - $100K,23.33


## Compute lambda values for a single lead:

In [4]:
lead = { 
        "Monthly Users": "50K - 100K",
        "Industry": "Technology",
        "Mkt Investment": "$300K - $400K",
    }
    
lambda_value = model.compute_lambda(lead)
lambda_value

80.48

## Removing features

In [5]:
model.remove_features(['Mkt Investment'])

In [6]:
model.describe_features()

{'Monthly Users': {'weight': 0.5, 'normalized_weight': 0.89}}
{'Monthly Users': {'weight': 0.5, 'normalized_weight': 0.89}, 'Industry': {'weight': 0.25, 'normalized_weight': 0.45}}


{'Monthly Users': {'weight': 0.5, 'normalized_weight': 0.89},
 'Industry': {'weight': 0.25, 'normalized_weight': 0.45}}

### Compute new lambda after removing a feature:

In [7]:
# df with both lambdas and delta_percent
df['new_lambda'] = df.apply(
    lambda row: model.compute_lambda(row.to_dict()),
    axis=1
)

df['delta_percent'] = (df['new_lambda'] / df['lambda'] - 1)*100

df


,Monthly Users,Industry,Mkt Investment,lambda,new_lambda,delta_percent
0,50K - 100K,Technology,$300K - $400K,80.48,50.0,-37.872763
1,50K - 100K,Retail,100k - $200K,51.19,55.0,7.442860
2,Up to 50K,Agriculture,50k - $100K,23.33,2.0,-91.427347


In [8]:
round(sum( # proof of normalized weights sum to 1
    [i.normalized_weight**2 for i in model.features]),
    model.round_decimals 
) 

1.0